In [ ]:
pip install --upgrade transformers accelerate bitsandbytes huggingface_hub dataset peft trl

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
import pandas as pd

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
df = pd.read_csv("/kaggle/input/mistral-fine-tune-data/output.csv")

In [5]:
df.shape

(3512, 2)

In [6]:
df.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [7]:
df.isna().sum()

Context     0
Response    4
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df.shape

(3508, 2)

In [10]:
df.duplicated().sum()

760

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df.shape

(2748, 2)

In [13]:
df.reset_index(drop=True,inplace=True)

In [14]:
df.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [15]:
import json

output_path = "output.json"

# Create a multiline json
record_dict = json.loads(df.to_json(orient="records"))
record_json = json.dumps(record_dict)

with open(output_path, 'w') as f:
    f.write(record_json)

In [16]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
from transformers import AutoTokenizer,AutoModelForCausalLM, BitsAndBytesConfig

In [18]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [19]:
tokenizer.pad_token = tokenizer.eos_token

In [20]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [23]:
import torch
llm = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype=torch.float32,device_map={"": 0}, quantization_config=quantization_config)

In [24]:
input_ids = tokenizer("How are you doing?", return_tensors="pt").input_ids
output = llm.generate(input_ids.to(device))
print(tokenizer.decode(output[0], skip_special_tokens=True))

KeyboardInterrupt: 

### Cleaning the dataset

In [25]:
from datasets import load_dataset,DatasetDict

In [26]:
dataset = load_dataset("json",data_files="/kaggle/working/output.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 2748
    })
})

In [28]:
dataset_split = dataset['train'].train_test_split(test_size=0.2)
dataset = DatasetDict({"train":dataset_split['train'],"validation":dataset_split['test']})

In [29]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 2198
    })
    validation: Dataset({
        features: ['Context', 'Response'],
        num_rows: 550
    })
})

In [30]:
import re
import unicodedata

def clean_text(text):
    # Normalize Unicode
    text = unicodedata.normalize("NFKC", text)

    # Remove HTML tags
    text = re.sub(r"<[^>]+>", "", text)

    # Remove control characters & excessive whitespace
    text = re.sub(r"[\x00-\x1F\x7F]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [31]:
def clean_row(example):
    example["Context"] = clean_text(example["Context"])
    example["Response"] = clean_text(example["Response"])
    return example

In [32]:
dataset = dataset.map(clean_row)

Map:   0%|          | 0/2198 [00:00<?, ? examples/s]

Map:   0%|          | 0/550 [00:00<?, ? examples/s]

In [33]:
def format(example):
    return {
        "text": f"<|user|>\n{example['Context']}\n<|assistant|>\n{example['Response']}"
    }

dataset = dataset.map(format)

Map:   0%|          | 0/2198 [00:00<?, ? examples/s]

Map:   0%|          | 0/550 [00:00<?, ? examples/s]

In [34]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response', 'text'],
        num_rows: 2198
    })
    validation: Dataset({
        features: ['Context', 'Response', 'text'],
        num_rows: 550
    })
})

In [35]:
from peft import LoraConfig, get_peft_model

In [36]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['q_proj','v_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(llm, lora_config)
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [37]:
from transformers import TrainingArguments,Trainer
from trl import SFTTrainer

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [38]:
training_args = TrainingArguments(
    output_dir = "./mistral_fine_tuned",
    eval_strategy = "epoch",
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 16,
    learning_rate = 2e-5,
    weight_decay = 0.01,
    num_train_epochs = 2,
    fp16 = True,
    save_strategy="epoch",
    logging_steps=10,
    report_to="none"
)

In [39]:
llm.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, o

In [40]:
trainer = SFTTrainer(
    model=llm,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=lora_config
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/2198 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/2198 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2198 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2198 [00:00<?, ? examples/s]

...

Adding EOS to eval dataset:   0%|          | 0/550 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/550 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (26820 > 2048). Running this sequence through the model will result in indexing errors


Truncating eval dataset:   0%|          | 0/550 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [41]:
torch.cuda.empty_cache()

In [42]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.372600,2.375336


TrainOutput(global_step=68, training_loss=2.402301283443675, metrics={'train_runtime': 2040.023, 'train_samples_per_second': 2.155, 'train_steps_per_second': 0.033, 'total_flos': 1.3874643346784256e+16, 'train_loss': 2.402301283443675})

In [43]:
llm

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, o

In [44]:
trainer.push_to_hub("TinyLlama-1.1B-Chat-v1.0/mental-health-counselling")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/28p07/mistral_fine_tuned/commit/9be146ee06ecc09aacc253e98d6384b820dfa53f', commit_message='TinyLlama-1.1B-Chat-v1.0/mental-health-counselling', commit_description='', oid='9be146ee06ecc09aacc253e98d6384b820dfa53f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/28p07/mistral_fine_tuned', endpoint='https://huggingface.co', repo_type='model', repo_id='28p07/mistral_fine_tuned'), pr_revision=None, pr_num=None)

In [53]:
from transformers import pipeline

question = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"
generator = pipeline("text-generation", model="28p07/mistral_fine_tuned", device="cuda")
output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
print(output["generated_text"])

Device set to use cuda


I understand that you are going through a difficult time, and I want to offer you some support and guidance. Here are some steps you can take to change your feelings of being worthless and to start focusing on your worth:

1. Identify the root cause of your feelings of worthlessness: think about what has happened to you that has made you feel this way. Have you been criticized or rejected by others? Have you experienced a loss or a failure? Have you been hurt by someone?

2. Practice self-compassion: be kind and gentle with yourself. Remember that you are a


In [54]:
question = "I am in love with a guy with whom my parents are not agree to marry me and my father is very strict and he does not know about it, my mother knowss about it and she taunts me,humiliates me every single day. Sometimes I am in a lot of stress and discomfort. Somedays had been too tough for my physical,mental and emotional health. I couldn't see any hope and I don't know what will happen in future. I don't know how to cope up with this situation and I can't survive without that guy."

In [56]:
output = generator([{"role": "user", "content": question}], max_new_tokens=512, return_full_text=False)[0]
print(output["generated_text"])

I understand your situation and can empathize with your feelings. It can be very challenging to navigate through a situation like this, especially when your parents are not on the same page as you.

first and foremost, it's essential to communicate with your parents about your relationship and your feelings. Explain to them that you love this person and that you want to be with them. Let them know that you're committed to this relationship and that you're willing to work through any issues that may arise.

if your parents are not willing to accept your relationship, it's essential to understand that they have their own reasons for not approving of it. It's essential to listen to their concerns and try to understand their perspective.

it's also essential to take care of yourself. This can mean seeking therapy, talking to friends or family members, or engaging in activities that bring you joy and help you feel more relaxed and at ease.

remember that this situation is unique to you and 